# OpenAI Real-time API - TEST 음성 FAQ 시스템 튜토리얼

이 노트북에서는 OpenAI의 Real-time API를 활용하여 **음성 기반 회원 탈퇴 시스템**을 단계별로 구현합니다.

## 학습 목표
1. CSV 기반 회원 데이터베이스 구축
2. Function Calling용 도구(Tool) 정의
3. OpenAI Real-time API 연결 및 세션 구성
4. 텍스트 기반 대화형 데모 실행

## 환경 안내
- **Google Colab**에서 실행 가능합니다 (마이크 불필요)
- 텍스트 입력/출력 모드로 동작합니다
- OpenAI API 키가 필요합니다 (Real-time API 접근 권한)

## 전체 아키텍처
```
사용자 입력 (텍스트) → Real-time API → 의도 파악
                                        ├─ 일반 응답 → 텍스트 출력
                                        └─ Function Call → 함수 실행 → 결과 반환 → 응답 생성
```

In [1]:
# 필요한 패키지를 설치합니다
# websockets는 Real-time API 연결에 필수입니다
!pip install -q "openai[realtime]" python-dotenv nest_asyncio websockets

zsh:1: command not found: pip


In [2]:
import os
import io
import csv
import json
import asyncio
import base64
from datetime import datetime
from typing import Optional
from openai import AsyncOpenAI

# Jupyter 환경에서 asyncio 이벤트 루프를 중첩 실행할 수 있게 합니다
import nest_asyncio
nest_asyncio.apply()

print("모든 라이브러리가 성공적으로 로드되었습니다.")

모든 라이브러리가 성공적으로 로드되었습니다.


In [3]:
# API 키 설정
# 방법 1: 직접 입력 (Colab 권장)
import getpass
OPENAI_API_KEY = getpass.getpass("OpenAI API 키를 입력하세요: ")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# 방법 2: .env 파일 사용 (로컬 환경)
# from dotenv import load_dotenv
# load_dotenv()
# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

client = AsyncOpenAI(api_key=OPENAI_API_KEY)
print("API 클라이언트가 생성되었습니다.")

API 클라이언트가 생성되었습니다.


---
## 2. 데이터 레이어: 회원 데이터베이스

실제 서비스에서는 데이터베이스(PostgreSQL, MongoDB 등)를 사용하지만,
이 튜토리얼에서는 **CSV 데이터를 인메모리**로 로드하여 간단한 회원 데이터베이스를 구현합니다.

### 회원 데이터 구조
| 필드 | 설명 | 예시 |
|------|------|------|
| member_id | 회원 고유 ID | M001 |
| name | 이름 | 김철수 |
| phone | 전화번호 | 010-1234-5678 |
| email | 이메일 | chulsoo@example.com |
| birth_date | 생년월일 | 1990-05-15 |
| status | 상태 | active / withdrawn |

In [4]:
# 회원 데이터를 인메모리로 생성합니다
MEMBER_CSV_DATA = """member_id,name,phone,email,birth_date,registered_at,status
M001,김철수,010-1234-5678,chulsoo@example.com,1990-05-15,2022-01-10,active
M002,이영희,010-2345-6789,younghee@example.com,1985-08-22,2021-06-15,active
M003,박민수,010-3456-7890,minsu@example.com,1992-12-03,2023-03-20,active
M004,최수진,010-4567-8901,sujin@example.com,1988-03-11,2020-11-05,active
M005,정호준,010-5678-9012,hojun@example.com,1995-07-28,2022-08-30,active
M006,강미래,010-6789-0123,mirae@example.com,1991-01-19,2021-12-25,withdrawn
M007,윤서연,010-7890-1234,seoyeon@example.com,1993-09-07,2023-01-15,active
M008,임재현,010-8901-2345,jaehyun@example.com,1987-11-30,2020-05-18,active
M009,한소영,010-9012-3456,soyoung@example.com,1994-04-25,2022-07-12,withdrawn
M010,오동건,010-0123-4567,donggun@example.com,1989-06-14,2021-09-08,withdrawn"""

# CSV를 파싱하여 회원 목록으로 변환합니다
reader = csv.DictReader(io.StringIO(MEMBER_CSV_DATA))
MEMBERS = list(reader)

print(f"총 {len(MEMBERS)}명의 회원 데이터가 로드되었습니다.\n")
print(f"{'ID':<6} {'이름':<8} {'전화번호':<16} {'상태':<10}")
print("-" * 45)
for m in MEMBERS:
    status_icon = "Active" if m["status"] == "active" else "Withdrawn"
    print(f"{m['member_id']:<6} {m['name']:<8} {m['phone']:<16} {status_icon}")

총 10명의 회원 데이터가 로드되었습니다.

ID     이름       전화번호             상태        
---------------------------------------------
M001   김철수      010-1234-5678    Active
M002   이영희      010-2345-6789    Active
M003   박민수      010-3456-7890    Active
M004   최수진      010-4567-8901    Active
M005   정호준      010-5678-9012    Active
M006   강미래      010-6789-0123    Withdrawn
M007   윤서연      010-7890-1234    Active
M008   임재현      010-8901-2345    Active
M009   한소영      010-9012-3456    Withdrawn
M010   오동건      010-0123-4567    Withdrawn


In [5]:
# 회원 관리 함수 정의
# 이 함수들은 AI가 Function Calling으로 호출하는 "도구(Tool)"입니다.

def search_member_by_name(name: str) -> dict:
    """이름으로 회원을 검색합니다."""
    found = [m for m in MEMBERS if m["name"] == name]

    if not found:
        return {"success": False, "message": f"'{name}' 님을 찾을 수 없습니다.", "member": None}

    member = found[0]
    if member["status"] == "withdrawn":
        return {"success": False, "message": f"'{name}' 님은 이미 탈퇴한 회원입니다.", "member": None}

    return {
        "success": True,
        "message": f"'{name}' 님의 정보를 찾았습니다.",
        "member": {
            "member_id": member["member_id"],
            "name": member["name"],
            "phone": member["phone"][-4:],
            "email": member["email"],
            "registered_at": member["registered_at"]
        }
    }


def verify_member(name: str, phone_last_4: str, birth_date: str) -> dict:
    """본인 인증을 수행합니다."""
    found = [m for m in MEMBERS if m["name"] == name]

    if not found:
        return {"success": False, "verified": False, "message": f"'{name}' 님을 찾을 수 없습니다.", "member_id": None}

    member = found[0]
    if member["status"] == "withdrawn":
        return {"success": False, "verified": False, "message": f"'{name}' 님은 이미 탈퇴한 회원입니다.", "member_id": None}

    # 전화번호 뒷 4자리 확인
    actual_phone_last_4 = member["phone"].replace("-", "")[-4:]
    if phone_last_4.replace("-", "")[-4:] != actual_phone_last_4:
        return {"success": True, "verified": False, "message": "전화번호가 일치하지 않습니다.", "member_id": None}

    # 생년월일 확인
    normalized_birth = birth_date.replace("-", "").replace(".", "").replace("/", "")
    actual_birth = member["birth_date"].replace("-", "")
    if normalized_birth != actual_birth:
        return {"success": True, "verified": False, "message": "생년월일이 일치하지 않습니다.", "member_id": None}

    return {"success": True, "verified": True, "message": "본인 인증이 완료되었습니다.", "member_id": member["member_id"]}


def process_withdrawal(member_id: str, reason: Optional[str] = None) -> dict:
    """회원 탈퇴를 처리합니다."""
    for member in MEMBERS:
        if member["member_id"] == member_id:
            if member["status"] == "withdrawn":
                return {"success": False, "message": "이미 탈퇴 처리된 회원입니다."}
            member["status"] = "withdrawn"
            return {
                "success": True,
                "message": f"{member['name']} 님의 회원 탈퇴가 완료되었습니다.",
                "withdrawn_at": datetime.now().isoformat()
            }
    return {"success": False, "message": "해당 회원을 찾을 수 없습니다."}


# 테스트
print("=== 함수 테스트 ===")
print("\n1. 회원 검색:")
print(json.dumps(search_member_by_name("김철수"), ensure_ascii=False, indent=2))

print("\n2. 본인 인증 (올바른 정보):")
print(json.dumps(verify_member("김철수", "5678", "19900515"), ensure_ascii=False, indent=2))

print("\n3. 본인 인증 (잘못된 전화번호):")
print(json.dumps(verify_member("김철수", "1111", "19900515"), ensure_ascii=False, indent=2))

=== 함수 테스트 ===

1. 회원 검색:
{
  "success": true,
  "message": "'김철수' 님의 정보를 찾았습니다.",
  "member": {
    "member_id": "M001",
    "name": "김철수",
    "phone": "5678",
    "email": "chulsoo@example.com",
    "registered_at": "2022-01-10"
  }
}

2. 본인 인증 (올바른 정보):
{
  "success": true,
  "verified": true,
  "message": "본인 인증이 완료되었습니다.",
  "member_id": "M001"
}

3. 본인 인증 (잘못된 전화번호):
{
  "success": true,
  "verified": false,
  "message": "전화번호가 일치하지 않습니다.",
  "member_id": null
}


---
## 3. Function Calling: 도구(Tool) 정의

OpenAI Real-time API에 등록할 **도구(Tool)의 JSON Schema**를 정의합니다.
AI 모델은 이 스키마를 보고, 사용자 요청에 적합한 함수를 **자동으로 호출**합니다.

### 도구 스키마 구조
```json
{
    "type": "function",
    "name": "함수이름",
    "description": "AI가 이해할 함수 설명",
    "parameters": {
        "type": "object",
        "properties": { ... },
        "required": ["필수_파라미터"]
    }
}
```

### Function Calling 흐름
```
1. 사용자: "김철수 회원 탈퇴해 주세요"
2. AI 판단 → search_member_by_name("김철수") 호출 결정
3. 서버에서 function_call 이벤트 수신
4. 우리 코드에서 실제 함수 실행 → 결과 반환
5. AI가 결과를 보고 다음 단계 진행 (본인 인증 요청 등)
```

In [6]:
# Function Calling을 위한 도구 스키마 정의
TOOLS = [
    {
        "type": "function",
        "name": "search_member_by_name",
        "description": "회원 이름으로 회원 정보를 검색합니다. 회원이 존재하는지 확인할 때 사용합니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {
                    "type": "string",
                    "description": "검색할 회원의 이름 (예: 김철수)"
                }
            },
            "required": ["name"]
        }
    },
    {
        "type": "function",
        "name": "verify_member",
        "description": "회원 본인 인증을 수행합니다. 이름, 전화번호 뒷 4자리, 생년월일로 인증합니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {"type": "string", "description": "회원 이름"},
                "phone_last_4": {"type": "string", "description": "전화번호 뒷 4자리 (예: 5678)"},
                "birth_date": {"type": "string", "description": "생년월일 (예: 19900515 또는 1990-05-15)"}
            },
            "required": ["name", "phone_last_4", "birth_date"]
        }
    },
    {
        "type": "function",
        "name": "process_withdrawal",
        "description": "본인 인증이 완료된 회원의 탈퇴를 처리합니다. 반드시 verify_member로 본인 인증을 먼저 완료해야 합니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "member_id": {"type": "string", "description": "탈퇴할 회원의 ID (예: M001)"},
                "reason": {"type": "string", "description": "탈퇴 사유"}
            },
            "required": ["member_id"]
        }
    }
]

# 함수 매핑 및 실행
FUNCTION_MAP = {
    "search_member_by_name": search_member_by_name,
    "verify_member": verify_member,
    "process_withdrawal": process_withdrawal
}

def execute_function(name: str, arguments: dict) -> dict:
    """Function calling 결과를 실행합니다."""
    if name in FUNCTION_MAP:
        return FUNCTION_MAP[name](**arguments)
    return {"error": f"Unknown function: {name}"}

print(f"{len(TOOLS)}개의 도구가 정의되었습니다:")
for tool in TOOLS:
    print(f"  - {tool['name']}: {tool['description'][:50]}...")

3개의 도구가 정의되었습니다:
  - search_member_by_name: 회원 이름으로 회원 정보를 검색합니다. 회원이 존재하는지 확인할 때 사용합니다....
  - verify_member: 회원 본인 인증을 수행합니다. 이름, 전화번호 뒷 4자리, 생년월일로 인증합니다....
  - process_withdrawal: 본인 인증이 완료된 회원의 탈퇴를 처리합니다. 반드시 verify_member로 본인 인증...


---
## 4. OpenAI Real-time API 연결

Real-time API는 **WebSocket 기반의 양방향 스트리밍 API**입니다.
이 섹션에서는 텍스트 모드로 연결하여 대화를 진행합니다.

> Colab에서는 마이크를 사용할 수 없으므로, **텍스트 입력 → 텍스트 응답** 방식으로 동작합니다.

### 주요 이벤트
| 이벤트 | 설명 |
|--------|------|
| `session.created` | WebSocket 세션 생성 완료 |
| `session.updated` | 세션 설정(instructions, tools 등) 변경 완료 |
| `response.text.delta` | AI 응답 텍스트 조각 (스트리밍) |
| `response.function_call_arguments.done` | AI가 함수 호출을 요청 |
| `response.done` | 하나의 응답이 완료됨 |
| `error` | 오류 발생 |

### 세션 설정 포인트
- `modalities`: `["text"]` (텍스트 전용) 또는 `["text", "audio"]` (음성 포함)
- `instructions`: AI의 역할과 행동 규칙 (시스템 프롬프트)
- `tools`: Function Calling용 도구 목록

In [7]:
# 세션 설정
SESSION_CONFIG = {
    "modalities": ["text"],  # 텍스트 전용 모드 (Colab 호환)
    "instructions": """당신은 TEST FAQ를 담당하는 챗봇입니다.
처음 인사할 때 "안녕하세요, TEST FAQ를 담당하는 챗봇입니다. 무엇을 도와드릴까요?"라고 말하세요.

회원 탈퇴를 원하는 경우 다음 절차를 따르세요:
1. 먼저 회원님의 성함을 여쭤봅니다.
2. search_member_by_name 함수로 회원 존재 여부를 확인합니다.
3. 본인 인증을 위해 전화번호 뒷 4자리와 생년월일을 여쭤봅니다.
4. verify_member 함수로 본인 인증을 수행합니다.
5. 인증 성공 시, 탈퇴 사유를 여쭤보고 process_withdrawal로 탈퇴를 처리합니다.

주의사항:
- 반드시 본인 인증을 완료한 후에만 탈퇴를 진행하세요.
- 생년월일은 8자리 숫자로 받아주세요 (예: 19900515)
- 친절하고 공손한 말투를 사용하세요.
- 한국어로 대화하세요.
- 짧고 간결하게 응답하세요.""",
    "tools": TOOLS
}

print("세션 설정이 준비되었습니다.")
print(f"  모달리티: {SESSION_CONFIG['modalities']}")
print(f"  등록된 도구: {len(SESSION_CONFIG['tools'])}개")

세션 설정이 준비되었습니다.
  모달리티: ['text']
  등록된 도구: 3개


In [8]:
# 단일 대화 턴을 처리하는 핵심 함수

async def async_input(prompt: str) -> str:
    """input()을 별도 스레드에서 실행하여 이벤트 루프를 차단하지 않습니다.
    이렇게 해야 WebSocket keepalive ping이 정상 처리됩니다."""
    return await asyncio.to_thread(input, prompt)


async def single_conversation_turn(user_message: str, connection):
    """사용자 메시지를 보내고 AI 응답을 받습니다."""

    # 1. 사용자 메시지를 대화에 추가
    await connection.conversation.item.create(
        item={
            "type": "message",
            "role": "user",
            "content": [{"type": "input_text", "text": user_message}]
        }
    )

    # 2. 응답 생성 요청
    await connection.response.create()

    # 3. 이벤트를 수신하며 응답을 처리
    response_text = ""
    async for event in connection:
        # 텍스트 응답 조각
        if event.type == "response.text.delta":
            response_text += event.delta
            print(event.delta, end="", flush=True)

        # 텍스트 응답 완료
        elif event.type == "response.text.done":
            print()  # 줄바꿈

        # Function Calling 요청
        elif event.type == "response.function_call_arguments.done":
            name = event.name
            arguments = json.loads(event.arguments)
            print(f"\n  [Function Call] {name}({json.dumps(arguments, ensure_ascii=False)})")

            # 함수 실행
            result = execute_function(name, arguments)
            print(f"  [Result] {json.dumps(result, ensure_ascii=False)}\n")

            # 결과를 대화에 추가
            await connection.conversation.item.create(
                item={
                    "type": "function_call_output",
                    "call_id": event.call_id,
                    "output": json.dumps(result, ensure_ascii=False)
                }
            )
            # AI가 함수 결과를 보고 이어서 응답
            await connection.response.create()

        # 전체 응답 완료
        elif event.type == "response.done":
            break

        # 오류
        elif event.type == "error":
            print(f"\n  [Error] {event.error.message}")
            break

    return response_text

print("single_conversation_turn() / async_input() 함수가 정의되었습니다.")

single_conversation_turn() / async_input() 함수가 정의되었습니다.


In [9]:
# API 연결 테스트

async def test_connection():
    """API 연결을 테스트합니다."""
    print("API 연결 테스트 중...")
    try:
        async with client.beta.realtime.connect(model="gpt-4o-mini-realtime-preview") as conn:
            await conn.session.update(session=SESSION_CONFIG)

            async for event in conn:
                if event.type == "session.created":
                    print("  세션 생성 완료")
                elif event.type == "session.updated":
                    print("  세션 설정 완료")
                    print("\nAPI 연결 테스트 성공!")
                    break
                elif event.type == "error":
                    print(f"  오류: {event.error.message}")
                    break
    except Exception as e:
        print(f"연결 실패: {e}")
        print("\nAPI 키와 Real-time API 접근 권한을 확인해주세요.")

await test_connection()

API 연결 테스트 중...
  세션 생성 완료
  세션 설정 완료

API 연결 테스트 성공!


---
## 5. 대화형 데모

아래 셀을 실행하면 텍스트로 AI와 대화할 수 있습니다.

### 테스트 시나리오
1. AI가 먼저 인사합니다
2. `회원 탈퇴하고 싶습니다` 라고 입력
3. AI가 이름을 물으면 → `김철수`
4. AI가 전화번호 뒷자리를 물으면 → `5678`
5. AI가 생년월일을 물으면 → `19900515`
6. AI가 탈퇴 사유를 물으면 → `개인 사정`

> `quit` 또는 `종료`를 입력하면 대화를 종료합니다.

In [10]:
# 자동 테스트 데모 (입력 없이 바로 실행)
# 미리 정의된 시나리오로 전체 흐름을 테스트합니다.

TEST_MESSAGES = [
    "회원 탈퇴하고 싶습니다",
    "김철수",
    "5678",
    "19900515",
    "개인 사정으로 탈퇴합니다",
]

async def auto_test_demo():
    """미리 정의된 메시지로 자동 테스트합니다."""
    print("=" * 50)
    print("  자동 테스트 모드")
    print(f"  {len(TEST_MESSAGES)}개의 메시지를 순서대로 전송합니다.")
    print("=" * 50)

    async with client.beta.realtime.connect(model="gpt-4o-mini-realtime-preview") as conn:
        await conn.session.update(session=SESSION_CONFIG)

        async for event in conn:
            if event.type == "session.updated":
                break

        # AI 첫 인사
        await conn.response.create()
        print("\nAI: ", end="")
        async for event in conn:
            if event.type == "response.text.delta":
                print(event.delta, end="", flush=True)
            elif event.type == "response.done":
                print("\n")
                break

        # 미리 정의된 메시지를 순서대로 전송
        for msg in TEST_MESSAGES:
            print(f"나: {msg}")
            print("AI: ", end="")
            await single_conversation_turn(msg, conn)
            print()
            await asyncio.sleep(1)  # 응답 간 1초 대기

    print("\n" + "=" * 50)
    print("  테스트 완료!")
    print("=" * 50)

await auto_test_demo()

  자동 테스트 모드
  5개의 메시지를 순서대로 전송합니다.

AI: 안녕하세요, TEST FAQ를 담당하는 챗봇입니다. 무엇을 도와드릴까요?

나: 회원 탈퇴하고 싶습니다
AI: 회원 탈퇴를 원하시는군요. 먼저 회원님의 성함을 알려주시겠어요?

나: 김철수
AI: 
  [Function Call] search_member_by_name({"name": "김철수"})
  [Result] {"success": true, "message": "'김철수' 님의 정보를 찾았습니다.", "member": {"member_id": "M001", "name": "김철수", "phone": "5678", "email": "chulsoo@example.com", "registered_at": "2022-01-10"}}


나: 5678
AI: '김철수' 님의 정보를 찾았습니다. 본인 인증을 위해 전화번호 뒷 4자리와 생년월일을 알려주실 수 있을까요?

나: 19900515
AI: 감사합니다. 생년월일도 알려주시겠어요? (예: 19900515)

나: 개인 사정으로 탈퇴합니다
AI: 
  [Function Call] verify_member({"name": "김철수", "phone_last_4": "5678", "birth_date": "19900515"})
  [Result] {"success": true, "verified": true, "message": "본인 인증이 완료되었습니다.", "member_id": "M001"}



  테스트 완료!


---
## 6. Gradio 웹 UI (노트북에서 바로 실행)

터미널 `input()` 대신 **Gradio 웹 인터페이스**로 대화할 수 있습니다.
아래 셀을 실행하면 노트북 안에 채팅 UI가 표시됩니다.

### 기능
- **텍스트 채팅**: 입력창에 텍스트를 입력하여 대화
- **파장 애니메이션**: AI 응답 중 아크 리액터가 파장처럼 울림
- **Function Calling**: 회원 검색/인증/탈퇴가 자동으로 동작
- **대화 내역**: 모든 대화가 채팅 형태로 표시
- **음성 출력**: AI가 응답하면 음성이 자동 재생됩니다

> **음성 입력(마이크)** 은 WebRTC가 필요하여 노트북에서는 지원되지 않습니다.
> 음성 입력이 필요하면 터미널에서 `python gradio_app/app.py`로 실행하세요.

In [ ]:
# Gradio 설치 (Colab에서는 이 셀을 실행하세요)
!pip install -q gradio numpy

In [ ]:
import gradio as gr
import numpy as np

SAMPLE_RATE = 24000  # Real-time API requires 24kHz

# ============================================================
# JARVIS CSS Theme
# ============================================================
JARVIS_CSS = """
.gradio-container {
    background: linear-gradient(180deg, #06080f 0%, #0a0f1a 40%, #080d18 100%) !important;
    color: #c0d8f0 !important;
    font-family: 'Segoe UI', 'Roboto', -apple-system, sans-serif !important;
}
.jarvis-grid-overlay {
    position: fixed; top:0; left:0; right:0; bottom:0;
    background-image:
        linear-gradient(rgba(0,212,255,0.025) 1px, transparent 1px),
        linear-gradient(90deg, rgba(0,212,255,0.025) 1px, transparent 1px);
    background-size: 60px 60px; pointer-events: none; z-index: 0;
}
.jarvis-scanline {
    position: fixed; left:0; right:0; height: 2px;
    background: linear-gradient(90deg, transparent, rgba(0,212,255,0.3), transparent);
    animation: scanmove 8s linear infinite; pointer-events: none; z-index: 9999;
}
@keyframes scanmove { 0%{top:-2px} 100%{top:100vh} }

.jarvis-header { text-align: center; padding: 20px 0 12px; }
.header-line {
    height: 1px; margin: 0 auto; width: 80%;
    background: linear-gradient(90deg, transparent, rgba(0,212,255,0.5) 50%, transparent);
}
.jarvis-title {
    font-family: 'Courier New', monospace; font-size: 28px; font-weight: 300;
    letter-spacing: 10px; color: #00d4ff; margin: 10px 0 2px;
    text-shadow: 0 0 30px rgba(0,212,255,0.6), 0 0 60px rgba(0,212,255,0.2);
}
.jarvis-subtitle {
    font-family: 'Courier New', monospace; font-size: 10px;
    letter-spacing: 4px; color: rgba(0,212,255,0.4); text-transform: uppercase; margin: 0 0 10px;
}

.block, .wrap, .contain, .panel, .form, .block.padded { background: transparent !important; border-color: rgba(0,212,255,0.1) !important; }

button.jarvis-connect-btn {
    font-family: 'Courier New', monospace !important; letter-spacing: 2px !important;
    text-transform: uppercase !important; font-size: 11px !important;
    background: linear-gradient(135deg, rgba(0,212,255,0.15), rgba(0,100,180,0.15)) !important;
    color: #00d4ff !important; border: 1px solid rgba(0,212,255,0.4) !important; border-radius: 2px !important;
}
button.jarvis-connect-btn:hover { box-shadow: 0 0 25px rgba(0,212,255,0.25) !important; }
button.jarvis-disconnect-btn {
    font-family: 'Courier New', monospace !important; letter-spacing: 2px !important;
    text-transform: uppercase !important; font-size: 11px !important;
    background: rgba(255,60,60,0.08) !important; color: #ff6b6b !important;
    border: 1px solid rgba(255,60,60,0.3) !important; border-radius: 2px !important;
}
button.jarvis-send-btn {
    font-family: 'Courier New', monospace !important; letter-spacing: 2px !important;
    text-transform: uppercase !important; font-size: 11px !important;
    background: linear-gradient(135deg, rgba(0,255,136,0.12), rgba(0,180,100,0.12)) !important;
    color: #00ff88 !important; border: 1px solid rgba(0,255,136,0.35) !important; border-radius: 2px !important;
}

/* Tabs */
.jarvis-tabs .tab-container[role="tablist"] {
    background: transparent !important; border-bottom: 1px solid rgba(0,212,255,0.12) !important;
    justify-content: center !important;
}
.jarvis-tabs .tab-container button[role="tab"] {
    background: transparent !important; color: rgba(192,216,240,0.4) !important;
    border: none !important; border-bottom: 2px solid transparent !important;
    padding: 14px 32px !important; font-family: 'Courier New', monospace !important;
    font-size: 13px !important; letter-spacing: 3px !important; text-transform: uppercase !important;
    border-radius: 0 !important;
}
.jarvis-tabs .tab-container button[role="tab"]:hover { color: rgba(0,212,255,0.7) !important; }
.jarvis-tabs .tab-container button[role="tab"].selected {
    color: #00d4ff !important; border-bottom: 2px solid #00d4ff !important;
    text-shadow: 0 0 15px rgba(0,212,255,0.5) !important;
}

/* Arc Reactor */
.arc-reactor-wrap { display: flex; flex-direction: column; align-items: center; justify-content: center; padding: 35px 10px 20px; }
.arc-reactor { width: 200px; height: 200px; position: relative; display: flex; align-items: center; justify-content: center; }
.arc-core { width: 28px; height: 28px; border-radius: 50%; position: absolute; z-index: 10; transition: all 0.5s; }
.arc-ring { position: absolute; border-radius: 50%; top: 50%; left: 50%; transform: translate(-50%,-50%); transition: border-color 0.5s; }
.arc-ring-1 { width: 65px; height: 65px; border: 2px solid rgba(0,212,255,0.5); border-color: rgba(0,212,255,0.6) transparent rgba(0,212,255,0.6) transparent; animation: arcspin 3s linear infinite; }
.arc-ring-2 { width: 100px; height: 100px; border: 1.5px dashed rgba(0,212,255,0.25); animation: arcspinrev 5s linear infinite; }
.arc-ring-3 { width: 135px; height: 135px; border: 1.5px solid transparent; border-color: transparent rgba(0,212,255,0.2) transparent rgba(0,212,255,0.2); animation: arcspin 8s linear infinite; }
.arc-ring-4 { width: 170px; height: 170px; border: 1px solid rgba(0,212,255,0.08); }

.arc-reactor.idle .arc-core { background: radial-gradient(circle, #b0e0ff, #00a8e0 40%, #005580); box-shadow: 0 0 25px rgba(0,170,230,0.5), 0 0 50px rgba(0,170,230,0.2); }
.arc-reactor.speaking .arc-core { background: radial-gradient(circle, #fff, #00ffaa 35%, #00aa66); box-shadow: 0 0 40px rgba(0,255,170,0.7), 0 0 80px rgba(0,255,170,0.3); animation: corebeat 0.6s ease-in-out infinite; }
.arc-reactor.speaking .arc-ring-1 { border-color: rgba(0,255,170,0.8) transparent rgba(0,255,170,0.8) transparent; animation: arcspin 1.2s linear infinite; }
.arc-reactor.speaking .arc-ring-2 { border-color: rgba(0,255,170,0.4); animation: arcspinrev 2s linear infinite; }
.arc-reactor.speaking .arc-ring-3 { border-color: transparent rgba(0,255,170,0.3) transparent rgba(0,255,170,0.3); animation: arcspin 3s linear infinite; }
.arc-reactor.speaking::before, .arc-reactor.speaking::after { content: ''; position: absolute; border-radius: 50%; border: 1px solid rgba(0,255,170,0.35); top: 50%; left: 50%; width: 100%; height: 100%; transform: translate(-50%,-50%); }
.arc-reactor.speaking::before { animation: reactorwave 1.8s ease-out infinite; }
.arc-reactor.speaking::after { animation: reactorwave 1.8s ease-out infinite 0.6s; }
.arc-reactor.disconnected .arc-core { background: radial-gradient(circle, #555, #333); box-shadow: 0 0 8px rgba(80,80,80,0.3); }
.arc-reactor.disconnected .arc-ring { border-color: rgba(80,80,80,0.15) !important; animation-play-state: paused !important; }
.reactor-label { margin-top: 18px; font-family: 'Courier New', monospace; font-size: 11px; letter-spacing: 5px; text-transform: uppercase; color: rgba(0,212,255,0.5); }
.reactor-label.active { color: #00ff88; text-shadow: 0 0 12px rgba(0,255,136,0.5); }

.log-section-label {
    font-family: 'Courier New', monospace; font-size: 10px; letter-spacing: 3px;
    color: rgba(0,212,255,0.4); text-transform: uppercase; text-align: center;
    padding: 12px 0 4px; border-top: 1px solid rgba(0,212,255,0.08); margin-top: 8px;
}

#jarvis-chat { background: rgba(6,8,15,0.7) !important; border: 1px solid rgba(0,212,255,0.12) !important; border-radius: 4px !important; position: relative; }
#jarvis-chat::before { content: ''; position: absolute; top: -1px; left: -1px; width: 18px; height: 18px; border-top: 2px solid rgba(0,212,255,0.5); border-left: 2px solid rgba(0,212,255,0.5); z-index: 5; pointer-events: none; }
#jarvis-chat::after { content: ''; position: absolute; bottom: -1px; right: -1px; width: 18px; height: 18px; border-bottom: 2px solid rgba(0,212,255,0.5); border-right: 2px solid rgba(0,212,255,0.5); z-index: 5; pointer-events: none; }

.jarvis-input textarea {
    background: rgba(6,8,15,0.8) !important; border: 1px solid rgba(0,212,255,0.18) !important;
    border-radius: 2px !important; color: #d0e4f5 !important; font-size: 14px !important;
    caret-color: #00d4ff !important; padding: 12px 16px !important;
}
.jarvis-input textarea:focus { border-color: rgba(0,212,255,0.45) !important; box-shadow: 0 0 20px rgba(0,212,255,0.08) !important; }
.jarvis-input textarea::placeholder { color: rgba(120,160,200,0.35) !important; }

.jarvis-audio { border: 1px solid rgba(0,212,255,0.12) !important; border-radius: 4px !important; }

label, .label-wrap span { color: rgba(0,212,255,0.5) !important; font-family: 'Courier New', monospace !important; font-size: 10px !important; letter-spacing: 2px !important; text-transform: uppercase !important; }

.jarvis-accordion { border: 1px solid rgba(0,212,255,0.1) !important; border-radius: 2px !important; background: rgba(6,8,15,0.4) !important; }
.jarvis-accordion .prose { color: #90b0cc !important; }
.jarvis-accordion th { background: rgba(0,212,255,0.08) !important; color: #00d4ff !important; font-family: 'Courier New', monospace !important; font-size: 11px !important; }
.jarvis-accordion td { border-color: rgba(0,212,255,0.08) !important; color: #90b0cc !important; }

.voice-notice { color: rgba(0,212,255,0.6) !important; text-align: center; padding: 30px; font-family: 'Courier New', monospace; }
.voice-notice code { background: rgba(0,212,255,0.1); padding: 2px 8px; border-radius: 3px; color: #00d4ff; }

::-webkit-scrollbar { width: 5px; }
::-webkit-scrollbar-track { background: transparent; }
::-webkit-scrollbar-thumb { background: rgba(0,212,255,0.25); border-radius: 3px; }

@keyframes arcspin { from { transform: translate(-50%,-50%) rotate(0deg); } to { transform: translate(-50%,-50%) rotate(360deg); } }
@keyframes arcspinrev { from { transform: translate(-50%,-50%) rotate(0deg); } to { transform: translate(-50%,-50%) rotate(-360deg); } }
@keyframes corebeat { 0%,100% { transform: scale(1); } 50% { transform: scale(1.25); } }
@keyframes reactorwave { 0% { width:100%; height:100%; opacity:0.6; } 100% { width:250%; height:250%; opacity:0; } }
footer { display: none !important; }
"""

# ============================================================
# HTML Templates
# ============================================================
HEADER_HTML = """
<div class="jarvis-grid-overlay"></div>
<div class="jarvis-scanline"></div>
<div class="jarvis-header">
    <div class="header-line"></div>
    <div class="jarvis-title">J.A.R.V.I.S.</div>
    <div class="jarvis-subtitle">TEST FAQ SYSTEM // REAL-TIME API v2.0</div>
    <div class="header-line"></div>
</div>
"""

_R = """<div class="arc-reactor-wrap"><div class="arc-reactor {s}">
<div class="arc-ring arc-ring-4"></div><div class="arc-ring arc-ring-3"></div>
<div class="arc-ring arc-ring-2"></div><div class="arc-ring arc-ring-1"></div>
<div class="arc-core"></div></div><div class="reactor-label {lc}">{lt}</div></div>"""

HTML_SPEAKING = _R.format(s="speaking", lc="active", lt="&#9679; RESPONDING")
HTML_IDLE = _R.format(s="idle", lc="", lt="&#9675; STANDBY")
HTML_DISCONNECTED = _R.format(s="disconnected", lc="", lt="&#9676; OFFLINE")
LOG_DIVIDER = '<div class="log-section-label">&#9662; COMMUNICATION LOG &#9662;</div>'

# ============================================================
# Real-time API Handler (text + audio playback)
# ============================================================
class NotebookRealtimeHandler:
    def __init__(self, api_key):
        self.client = AsyncOpenAI(api_key=api_key)
        self.connection = None
        self.is_connected = False
        self.is_speaking = False
        self.chat_history = []
        self.audio_output_buffer = []
        self.transcript_buffer = ""
        self._event_task = None
        self._context_manager = None

    async def connect(self):
        self._context_manager = self.client.beta.realtime.connect(model="gpt-4o-mini-realtime-preview")
        self.connection = await self._context_manager.__aenter__()
        self.is_connected = True
        await self.connection.session.update(session={
            "modalities": ["text", "audio"],
            "instructions": SESSION_CONFIG["instructions"],
            "voice": "alloy",
            "input_audio_format": "pcm16",
            "output_audio_format": "pcm16",
            "input_audio_transcription": {"model": "whisper-1"},
            "turn_detection": {
                "type": "server_vad",
                "threshold": 0.95,
                "prefix_padding_ms": 200,
                "silence_duration_ms": 1200
            },
            "tools": TOOLS
        })
        async for event in self.connection:
            if event.type == "session.updated":
                break
        await self.connection.response.create()
        self._event_task = asyncio.create_task(self._process_events())

    async def disconnect(self):
        self.is_connected = False
        if self._event_task:
            self._event_task.cancel()
            try: await self._event_task
            except asyncio.CancelledError: pass
        if self._context_manager:
            try: await self._context_manager.__aexit__(None, None, None)
            except: pass
        self.connection = None

    async def send_text(self, text):
        if not self.is_connected: return
        self.chat_history.append(("user", text))
        await self.connection.conversation.item.create(
            item={"type": "message", "role": "user",
                  "content": [{"type": "input_text", "text": text}]})
        await self.connection.response.create()

    def get_and_clear_audio_output(self):
        if not self.audio_output_buffer:
            return None
        combined = b"".join(self.audio_output_buffer)
        self.audio_output_buffer.clear()
        return combined

    async def _process_events(self):
        try:
            async for event in self.connection:
                if event.type == "response.audio.delta":
                    self.is_speaking = True
                    audio_bytes = base64.b64decode(event.delta)
                    self.audio_output_buffer.append(audio_bytes)
                elif event.type == "response.audio.done":
                    self.is_speaking = False
                elif event.type == "response.audio_transcript.delta":
                    self.transcript_buffer += event.delta
                elif event.type == "response.audio_transcript.done":
                    if self.transcript_buffer:
                        self.chat_history.append(("assistant", self.transcript_buffer))
                        self.transcript_buffer = ""
                elif event.type == "response.text.delta":
                    self.transcript_buffer += event.delta
                elif event.type == "response.text.done":
                    if self.transcript_buffer:
                        self.chat_history.append(("assistant", self.transcript_buffer))
                        self.transcript_buffer = ""
                elif event.type == "conversation.item.input_audio_transcription.completed":
                    if hasattr(event, "transcript") and event.transcript:
                        self.chat_history.append(("user", event.transcript))
                elif event.type == "input_audio_buffer.speech_started":
                    self.is_speaking = False
                    self.audio_output_buffer.clear()
                elif event.type == "response.function_call_arguments.done":
                    name = event.name
                    arguments = json.loads(event.arguments)
                    self.chat_history.append(("system", f"[Function: {name}({arguments})]"))
                    result = execute_function(name, arguments)
                    await self.connection.conversation.item.create(
                        item={"type": "function_call_output", "call_id": event.call_id,
                              "output": json.dumps(result, ensure_ascii=False)})
                    await self.connection.response.create()
                elif event.type == "error":
                    self.chat_history.append(("system", f"Error: {event.error.message}"))
        except asyncio.CancelledError: pass
        except Exception as e:
            self.chat_history.append(("system", f"Connection error: {str(e)}"))
            self.is_connected = False

# ============================================================
# Gradio Event Handlers
# ============================================================
nb_handler = None

async def on_connect():
    global nb_handler
    nb_handler = NotebookRealtimeHandler(OPENAI_API_KEY)
    try:
        await nb_handler.connect()
        await asyncio.sleep(2)
        return (_fmt(), HTML_IDLE, gr.update(interactive=False), gr.update(interactive=True))
    except Exception as e:
        return ([{"role":"assistant","content":f"Connection failed: {e}"}],
                HTML_DISCONNECTED, gr.update(interactive=True), gr.update(interactive=False))

async def on_disconnect():
    global nb_handler
    if nb_handler: await nb_handler.disconnect(); nb_handler = None
    return ([], HTML_DISCONNECTED, gr.update(interactive=True), gr.update(interactive=False))

async def on_send(text):
    if not nb_handler or not nb_handler.is_connected or not text.strip():
        return _fmt(), ""
    await nb_handler.send_text(text)
    await asyncio.sleep(1.5)
    return _fmt(), ""

def on_tick():
    if not nb_handler or not nb_handler.is_connected:
        return [], None, HTML_DISCONNECTED
    audio_bytes = nb_handler.get_and_clear_audio_output()
    audio_out = None
    if audio_bytes:
        pcm = np.frombuffer(audio_bytes, dtype=np.int16)
        audio_out = (SAMPLE_RATE, pcm)
    status = HTML_SPEAKING if nb_handler.is_speaking else HTML_IDLE
    return _fmt(), audio_out, status

def _fmt():
    if not nb_handler: return []
    msgs = []
    for role, content in nb_handler.chat_history:
        if role == "user": msgs.append({"role":"user","content":content})
        elif role == "assistant": msgs.append({"role":"assistant","content":content})
        elif role == "system": msgs.append({"role":"assistant","content":f"[SYS] {content}"})
    return msgs

# ============================================================
# Build JARVIS UI with VOICE / TEXT tabs
# ============================================================
with gr.Blocks(css=JARVIS_CSS) as demo:
    gr.HTML(HEADER_HTML)

    with gr.Row():
        conn_btn = gr.Button("CONNECT", variant="primary", scale=1, elem_classes=["jarvis-connect-btn"])
        disc_btn = gr.Button("DISCONNECT", variant="stop", scale=1, interactive=False, elem_classes=["jarvis-disconnect-btn"])

    # Arc Reactor - centered
    status_html = gr.HTML(value=HTML_DISCONNECTED)

    # Tabs: Voice / Text
    with gr.Tabs(elem_classes=["jarvis-tabs"]):
        with gr.Tab("VOICE MODE"):
            gr.HTML("""<div class="voice-notice">
                <p>&#127897; 음성 모드는 WebRTC가 필요하여 노트북에서는 지원되지 않습니다.</p>
                <p>음성 모드를 사용하려면 터미널에서 독립 실행하세요:</p>
                <p><code>python gradio_app/app.py</code></p>
            </div>""")
        with gr.Tab("TEXT MODE"):
            with gr.Row():
                txt = gr.Textbox(placeholder="Enter message... (press Enter)", label="MESSAGE", scale=5, elem_classes=["jarvis-input"])
                send = gr.Button("TRANSMIT", variant="primary", scale=1, elem_classes=["jarvis-send-btn"])

    # Hidden audio output for TEXT MODE playback
    audio_output = gr.Audio(label="AI OUTPUT", autoplay=True, visible=False)

    # Communication Log - below
    gr.HTML(LOG_DIVIDER)
    chatbot = gr.Chatbot(label="COMMUNICATION LOG", height=300, elem_id="jarvis-chat", type="messages")

    timer = gr.Timer(value=0.5)

    with gr.Accordion("TEST MEMBER DATABASE", open=False, elem_classes=["jarvis-accordion"]):
        gr.Markdown(
            "| NAME | PHONE | DOB | STATUS |\n|------|-------|-----|--------|\n"
            "| Kim Chulsu | 010-1234-5678 | 1990-05-15 | ACTIVE |\n"
            "| Lee Younghee | 010-2345-6789 | 1985-08-22 | ACTIVE |\n"
            "| Park Minsu | 010-3456-7890 | 1992-12-03 | ACTIVE |\n"
            "\n**TEST**: Name `김철수` / Last 4 `5678` / DOB `19900515`")

    # Event wiring
    conn_btn.click(fn=on_connect, outputs=[chatbot, status_html, conn_btn, disc_btn])
    disc_btn.click(fn=on_disconnect, outputs=[chatbot, status_html, conn_btn, disc_btn])
    txt.submit(fn=on_send, inputs=[txt], outputs=[chatbot, txt])
    send.click(fn=on_send, inputs=[txt], outputs=[chatbot, txt])
    timer.tick(fn=on_tick, outputs=[chatbot, audio_output, status_html])

demo.launch(inline=True, share=False)